# Euro 2024 - Pass Networks

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams

import urllib
from PIL import Image

from mplsoccer import Pitch, VerticalPitch, FontManager, Sbopen

In [2]:
def shot_jointgrid(df):
    
    # Ensure relevant columns are numeric and handle non-numeric values
    df['x'] = pd.to_numeric(df['x'], errors='coerce')
    df['y'] = pd.to_numeric(df['y'], errors='coerce')
    df['is_shot'] = df['is_shot'].astype(bool)
    
    # Filtering to include only the shots
    df_shots = df[df.is_shot == True].copy()
    
    # Subseting the two teams
    team1, team2 = df_shots.team.unique()
    df_team1 = df_shots[df_shots.team == team1].copy()
    df_team2 = df_shots[df_shots.team == team2].copy()
    
    # setup the mplsoccer StatsBomb Pitches
    # note not much padding around the pitch so the marginal axis are tight to the pitch
    # if you are using a different goal type you will need to increase the padding to see the goals
    pitch = Pitch(pad_top=0.05, pad_right=0.05, pad_bottom=0.05, pad_left=0.05, 
                  line_zorder=2, pitch_type='opta', line_color='#efefef', pitch_color='#1B1E22')
    
    # Usually in football, the data is collected so the attacking direction is left to right.
    # We can shift the coordinates via: new_x_coordinate = right_side - old_x_coordinate
    # This is helpful for having one team shots on the left of the pitch and the other on the right
    df_team1['x'] = pitch.dim.right - df_team1.x
    
    fig, axs = pitch.jointgrid(figheight=10, left=None, bottom=0.075, grid_height=0.8,
                           axis=False,  # turn off title/ endnote/ marginal axes
                           # plot without endnote/ title axes
                           endnote_height=0, title_height=0)
    
    fig.set_facecolor('#1B1E22')
    
    # Define custom colormaps
    reds = LinearSegmentedColormap.from_list('custom_reds', [(1, 0.9, 0.9, 0.3), (1, 0, 0, 1)])
    blues = LinearSegmentedColormap.from_list('custom_blues', [(0.9, 0.9, 1, 0.3), (0, 0, 1, 1)])

    # plot the hexbins
    hex1 = pitch.hexbin(df_team1.x, df_team1.y, ax=axs['pitch'],
                        edgecolors=pitch.line_color, cmap='Reds')
    hex2 = pitch.hexbin(df_team2.x, df_team2.y, ax=axs['pitch'],
                        edgecolors=pitch.line_color, cmap='Blues')
    
    # normalize the values so the colors depend on the minimum/ value for both teams
    # this ensures that darker colors mean more shots relative to both teams
    vmin = min(hex1.get_array().min(), hex2.get_array().min())
    vmax = max(hex1.get_array().max(), hex2.get_array().max())
    hex1.set_clim(vmin=vmin, vmax=vmax)
    hex2.set_clim(vmin=vmin, vmax=vmax)
    
    # Define colors
    red = 'red'
    blue = 'blue'
    
    # plot kdeplots on the marginals
    team1_hist_y = sns.kdeplot(y=df_team1.y, ax=axs['left'], color=red, fill=True, alpha=0.3)
    team1_hist_x = sns.kdeplot(x=df_team1.x, ax=axs['top'], color=red, fill=True, alpha=0.3)
    team2_hist_x = sns.kdeplot(x=df_team2.x, ax=axs['top'], color=blue, fill=True, alpha=0.3)
    team2_hist_y = sns.kdeplot(y=df_team2.y, ax=axs['right'], color=blue, fill=True, alpha=0.9)
    txt1 = axs['pitch'].text(x=15, y=90, s=team1, color=red,
                             ha='center', va='center', fontsize=30, alpha=0.7)
    txt2 = axs['pitch'].text(x=85, y=90, s=team2, color=blue,
                             ha='center', va='center', fontsize=30, alpha=0.7)
    
    


shot_jointgrid(final_df)
    

NameError: name 'final_df' is not defined

In [ ]:
def xT_momentum(df): 
    
    # Subseting the two teams
    team1, team2 = df.team.unique()
    
    # Keeping only specific Team records
    df = df.loc[(df['type_display_name']=='Pass') & (df['outcome_type_display_name']=='Successful')]
    df.reset_index(inplace=True)
    
    # Simulating xT scores for each section of the field
    # Importing from a different directory
    xT = pd.read_csv('/Users/enzovillafuerte/Desktop/Futbol Analytics Club/AGREF/xT_Grid.csv', header=None)
    
    # Converting values into array
    xT = np.array(xT)
    xT_rows, xT_cols = xT.shape
    
    # Categorizing each record in a bin for starting point and ending point
    df['x1_bin'] = pd.cut(df['x'], bins = xT_cols, labels=False)
    df['y1_bin'] = pd.cut(df['y'], bins = xT_rows, labels=False)

    df['x2_bin'] = pd.cut(df['end_x'], bins = xT_cols, labels=False)
    df['y2_bin'] = pd.cut(df['end_y'], bins = xT_rows, labels=False)
    
    # Defining start zone and end zone values of passes (kinda like x,y coordinates in a map plot)
    df['start_zone_value'] = df[['x1_bin', 'y1_bin']].apply(lambda x: xT[x[1]][x[0]],axis=1)
    df['end_zone_value'] = df[['x2_bin', 'y2_bin']].apply(lambda x: xT[x[1]][x[0]],axis=1)

    # The difference of end_zone and start_zone is the expected threat value for the action (pass) - not accounting for dribble xT here
    # Value can be negative or positive (progressive)
    df['Pass xT'] = df['end_zone_value'] - df['start_zone_value']
    # Progressive xT measures progressive passes
    df['Progressive xT'] = ''
    
    # Iterating and filling values for Progressive xT
    counter = 0 

    while counter < len(df):
        if df['Pass xT'][counter] > 0:
            df['Progressive xT'][counter] = df['Pass xT'][counter]
        else:
            df['Progressive xT'][counter] = 0.00
        counter += 1
        
    # Grouping the df by minute and team to calculate the average xT per minute for each team
    df['minute'] = df['minute'].astype(int)  # Ensure minute column is numeric
    avg_xT_per_minute = df.groupby(['minute', 'team'])['Pass xT'].mean().unstack()
    
    # Plot the Match Momentum by xT Chart
    fig, ax = plt.subplots(figsize=(11, 16))
    
    # Plot double-sided bar charts for home and away teams
    for team in avg_xT_per_minute.columns:
        if team == team1:
            ax.bar(avg_xT_per_minute.index, avg_xT_per_minute[team], color='blue', alpha=0.5, label=team)
        elif team == team2:
            ax.bar(avg_xT_per_minute.index, -avg_xT_per_minute[team], color='red', alpha=0.5, label=team)
            
    # Add labels and title
    ax.set_xlabel('Minute')
    ax.set_ylabel('Average xT per Minute')
    ax.set_title('Match Momentum by xT')
    
    # Add legend and adjust y-axis limits for better visualization
    ax.legend(loc='upper left')
    ax.set_ylim(bottom=-max(avg_xT_per_minute.max().abs()) * 1.1, top=max(avg_xT_per_minute.max().abs()) * 1.1)
            
    
        
    return df.head(3)





xT_momentum(final_df)
        